# Pytorch Quantization for RESNET

- Train a floating point model or load a pre-trained floating point model.
- Move the model to CPU and switch model to training mode.
- Apply layer fusion.
- Switch model to evaluation mode, check if the layer fusion results in correct model, and switch back to training mode.
- Apply torch.quantization.QuantStub() and torch.quantization.QuantStub() to the inputs and outputs, respectively.
- Specify quantization configurations, such as symmetric quantization or asymmetric quantization, etc.
- Prepare quantization model for quantization aware training.
- Move the model to CUDA and run quantization aware training using CUDA.
- Move the model to CPU and convert the quantization aware trained floating point model to quantized integer model.
- [Optional] Verify accuracies and inference performance gain.
- Save the quantized integer model.

In [1]:
import os
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import time
import copy
import numpy as np
from fedlern.train_utils import *
from fedlern.quant_utils import *
from fedlern.models.resnet_v2 import *


# Static Quantization

In [2]:
random_seed = 0
num_classes = 10
stats = (0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784)

cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")
set_random_seeds(random_seed=random_seed)


In [3]:
model_dir = "saved_models"
model_filename = 'resnet18v2_cifar10.pt'
quantized_model_filename = "resnet18_quantized_cifar10pytorch.pt"
model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)

In [4]:


# Create an untrained model.
#model = create_model(num_classes=num_classes)
model = ResNet18()

train_loader, test_loader = prepare_dataloader_cifar(num_workers=8, train_batch_size=128, eval_batch_size=256, stats=stats)


Files already downloaded and verified


In [5]:

# # Train model.
# model = train_model(model=model, train_loader=train_loader, test_loader=test_loader, device=cuda_device)
# # Save model.
# save_model(model=model, model_dir=model_dir, model_filename=model_filename)

# Load a pretrained model.
model = load_model(model=model, model_filepath=model_filepath, device=cuda_device)
# Move the model to CPU since static quantization does not support CUDA currently.
model.to(cpu_device)
# Make a copy of the model for layer fusion
fused_model = copy.deepcopy(model)

In [6]:
# The model has to be switched to training mode before any layer fusion.
# Otherwise the quantization aware training will not work correctly.
fused_model.eval()
print(fused_model)
# Fuse the model in place rather manually.
# Fuse the first Conv2d, BatchNorm2d, and ReLU layers
fused_model = torch.quantization.fuse_modules(fused_model, [["conv1", "bn1", "relu"]], inplace=False)
# Fuse the remaining Conv2d and BatchNorm2d layers in the ResNet blocks
for name, module in fused_model.named_modules():
    if isinstance(module, nn.Sequential):
        for i in range(len(module)):
            if isinstance(module[i], BasicBlock) or isinstance(module[i], Bottleneck):
                module[i] = torch.quantization.fuse_modules(module[i], [["conv1", "bn1", "relu"], ["conv2", "bn2"]], inplace = True)


# Print FP32 model.
print("FP32", model)
# Print fused model.
print("FUSED", fused_model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (

In [7]:
model.eval()
fused_model.eval()

# Model and fused model should be equivalent.
#assert model_equivalence(model_1=model, model_2=fused_model, device=cpu_device, rtol=1e-03, atol=1e-06, num_tests=100, input_size=(1,3,32,32)), "Fused model is not equivalent to the original model!"


ResNet(
  (conv1): ConvReLU2d(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (bn1): Identity()
  (relu): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (bn1): Identity()
      (relu): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (bn1): Identity()
      (relu): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 

In [8]:

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
quantized_model = QuantizedResNet(model_fp32=fused_model)
# Using un-fused model will fail.
# Because there is no quantized layer implementation for a single batch normalization layer.
# quantized_model = QuantizedResNet18(model_fp32=model)
# Select quantization schemes from 
# https://pytorch.org/docs/stable/quantization-support.html
quantization_config = torch.quantization.get_default_qconfig("fbgemm")
# Custom quantization configurations
# quantization_config = torch.quantization.default_qconfig
# quantization_config = torch.quantization.QConfig(activation=torch.quantization.MinMaxObserver.with_args(dtype=torch.quint8), weight=torch.quantization.MinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))

quantized_model.qconfig = quantization_config

# Print quantization configurations
print(quantized_model.qconfig)

torch.quantization.prepare_qat(quantized_model, inplace=True)

# # Use training data for calibration.
print("Training QAT Model...")
quantized_model.train()
train_model(model=quantized_model, train_loader=train_loader, test_loader=test_loader, device=cuda_device, learning_rate=1e-3, num_epochs=10)
quantized_model.to(cpu_device)

quantized_model = torch.quantization.convert(quantized_model, inplace=True)

# Using high-level static quantization wrapper
# The above steps, including torch.quantization.prepare, calibrate_model, and torch.quantization.convert, are also equivalent to
# quantized_model = torch.quantization.quantize(model=quantized_model, run_fn=calibrate_model, run_args=[train_loader], mapping=None, inplace=False)

quantized_model.eval()

# Print quantized model.
print("Quantized", quantized_model)

# Save quantized model.
# save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)

# Load quantized model.
# quantized_jit_model = load_torchscript_model(model_filepath=quantized_model_filepath, device=cpu_device)



QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})
Training QAT Model...


/home/isaac/.local/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


TypeError: train_model() got an unexpected keyword argument 'learning_rate'

In [ ]:
_, fp32_eval_accuracy = evaluate_model(model=model, test_loader=test_loader, device=cpu_device, criterion=None)
_, int8_eval_accuracy = evaluate_model(model=quantized_model, test_loader=test_loader, device=cpu_device, criterion=None)

# Skip this assertion since the values might deviate a lot.
# assert model_equivalence(model_1=model, model_2=quantized_jit_model, device=cpu_device, rtol=1e-01, atol=1e-02, num_tests=100, input_size=(1,3,32,32)), "Quantized model deviates from the original model too much!"

print("FP32 evaluation accuracy: {:.3f}".format(fp32_eval_accuracy))
print("INT8 evaluation accuracy: {:.3f}".format(int8_eval_accuracy))

fp32_cpu_inference_latency = measure_inference_latency(model=model, device=cuda_device, input_size=(1,3,32,32), num_samples=100)
int8_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
int8_jit_cpu_inference_latency = measure_inference_latency(model=quantized_model, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
fp32_gpu_inference_latency = measure_inference_latency(model=model, device=cuda_device, input_size=(1,3,32,32), num_samples=100)

print("FP32 CPU Inference Latency: {:.2f} ms / sample".format(fp32_cpu_inference_latency * 1000))
print("FP32 CUDA Inference Latency: {:.2f} ms / sample".format(fp32_gpu_inference_latency * 1000))
print("INT8 CPU Inference Latency: {:.2f} ms / sample".format(int8_cpu_inference_latency * 1000))
print("INT8 JIT CPU Inference Latency: {:.2f} ms / sample".format(int8_jit_cpu_inference_latency * 1000))


FP32 evaluation accuracy: 0.927
INT8 evaluation accuracy: 0.100
FP32 CPU Inference Latency: 2.93 ms / sample
FP32 CUDA Inference Latency: 2.93 ms / sample
INT8 CPU Inference Latency: 3.57 ms / sample
INT8 JIT CPU Inference Latency: 5.02 ms / sample
